---
# Explicación del Script `preprocess_and_save_simplified_model.py`

Este script Python (`preprocess_and_save_simplified_model.py`) es fundamental para el proyecto, ya que se encarga de preparar los datos de cangrejos, aplicar transformaciones necesarias y entrenar un **modelo de regresión lineal simplificado**. Posteriormente, guarda los modelos entrenados y sus métricas para su uso futuro, por ejemplo, en una aplicación Streamlit.

---

## 1. Propósito General del Script

El objetivo principal de este script es:
* Cargar los datos originales de cangrejos.
* **Filtrar los datos** para un rango de peso específico, que será el usado para el entrenamiento del modelo.
* Aplicar la **transformación Box-Cox** a las características y variables objetivo para asegurar que se ajusten a los supuestos de la regresión lineal (normalidad y homocedasticidad).
* Entrenar un **modelo de regresión lineal** para predecir los pesos de la carne, vísceras y caparazón basándose únicamente en el peso total del cangrejo.
* Calcular y almacenar las **métricas de evaluación** del modelo (MAE y R²) para los conjuntos de entrenamiento y prueba.
* **Guardar** los objetos transformadores (lambdas de Box-Cox) y los modelos entrenados para que puedan ser cargados y utilizados en otras partes del proyecto sin necesidad de re-entrenar.

---

## 2. Configuración y Estructura del Script

El script está bien estructurado con una sección de configuración clara y pasos lógicos:

* **Variables de Configuración:** Define rutas de archivos, nombres de carpetas (`data_sets`, `models`), nombres de columnas (`new_weight`, `Shucked Weight`, etc.) y los rangos de peso para el entrenamiento.
* **Creación de Carpetas:** Asegura que las carpetas `data_sets` y `models` existan antes de intentar guardar archivos en ellas, evitando errores.
* **Manejo de Errores:** Incluye un bloque `try-except` para la carga del CSV, lo que lo hace más robusto en caso de que el archivo `crabs.csv` no se encuentre.

---

## 3. Preprocesamiento de Datos Clave

### A. Carga y Copia del DataFrame
El script carga el archivo `crabs.csv` y crea una copia (`df_transformed`). Trabajar con una copia es una buena práctica para asegurar que el DataFrame original no se modifique accidentalmente.

### B. Filtrado de Datos para Entrenamiento (¡Novedad importante!)
Una de las modificaciones más relevantes es el **filtrado de los datos de entrenamiento**. Se seleccionan solo aquellas filas donde el `new_weight` (peso total) del cangrejo está entre `MIN_WEIGHT_TRAIN` (5.0 gramos) y `MAX_WEIGHT_TRAIN` (85.0 gramos).

* **¿Por qué es importante este filtrado?**
    * **Relevancia del Modelo:** Es probable que los modelos de regresión sean más precisos para el rango de pesos más comunes o más interesantes para el análisis. Entrenar el modelo con datos fuera de este rango podría introducir ruido o distorsionar las relaciones lineales para el rango de interés principal.
    * **Evitar Extrapolación Extrema:** Al limitar el rango de entrenamiento, se busca que el modelo no intente "aprender" de datos atípicos o extremos que podrían no seguir la misma relación lineal, mejorando la generalización dentro del rango definido.
    * **Optimización del Rendimiento:** Reducir el tamaño del dataset de entrenamiento si hay muchos datos irrelevantes puede, en ocasiones, acelerar el proceso y enfocar el aprendizaje.

### C. Transformación Box-Cox
Este es un paso crucial que aborda los supuestos de los modelos lineales:

* Se itera sobre las columnas especificadas (`new_weight`, `Shucked Weight`, `Viscera Weight`, `Shell Weight`).
* Para cada columna, se aplica la **transformación Box-Cox** para intentar que su distribución se acerque a una normal y estabilizar su varianza.
* **Manejo de Valores No Positivos:** Si alguna columna contiene valores menores o iguales a cero (requisito de Box-Cox), se añade un pequeño `offset` para hacerlos positivos antes de la transformación.
* **Captura de Lambdas:** El script almacena el valor `lambda` calculado para cada transformación. Estos `lambdas` son vitales para realizar la **transformación inversa** de las predicciones del modelo, es decir, convertir los valores predichos (que están en escala Box-Cox) de vuelta a la escala original de gramos. Sin ellos, las predicciones no serían interpretables.
* **Guardado de Lambdas:** El diccionario `boxcox_lambdas` se guarda como un archivo `.joblib` para ser cargado posteriormente en la aplicación.

---

## 4. Entrenamiento y Evaluación del Modelo

### A. Preparación de Datos para el Modelo
* Se seleccionan la feature transformada (`new_weight_boxcox`) y los targets transformados (`Shucked Weight_boxcox`, etc.).
* Se manejan los valores `NaN` para asegurar que el entrenamiento se realice con datos completos.
* Los datos se dividen en conjuntos de **entrenamiento (80%) y prueba (20%)** usando `train_test_split` con `random_state=42` para reproducibilidad.

### B. Entrenamiento de Modelos Individuales
* Se entrena un **modelo de `LinearRegression` por separado** para cada una de las tres variables objetivo (`Shucked Weight`, `Viscera Weight`, `Shell Weight`). Esto significa que tendrás tres modelos de regresión lineal, cada uno optimizado para predecir una parte específica del cangrejo.
* Los modelos entrenados se almacenan en el diccionario `trained_models`.

### C. Evaluación de Métricas
* Para cada modelo, se calculan el **Error Absoluto Medio (MAE)** y el **Coeficiente de Determinación (R²)** tanto para el conjunto de entrenamiento como para el de prueba.
* **MAE** ($$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$): Mide el error promedio absoluto entre las predicciones y los valores reales. Un MAE más bajo indica un mejor rendimiento.
* **R²** ($$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$): Representa la proporción de la varianza en la variable dependiente que es predecible a partir de la(s) variable(s) independiente(s). Un R² más cercano a 1 indica que el modelo explica una mayor proporción de la variabilidad.
* Estas métricas son cruciales para entender el rendimiento del modelo y se guardan en el diccionario `all_model_metrics`.

---

## 5. Guardado de Activos del Modelo

Finalmente, el script serializa y guarda los objetos clave:
* **`trained_models`:** El diccionario que contiene los tres modelos de regresión lineal entrenados.
* **`all_model_metrics`:** El diccionario que almacena las métricas de evaluación de cada modelo.

Ambos se guardan usando `joblib`, una librería eficiente para serializar objetos Python grandes, lo que permite cargarlos rápidamente en la aplicación Streamlit sin necesidad de re-entrenar los modelos cada vez.

---

## 6. ¿Cómo Influyó este Script en el Proyecto?

Este script es el **"cerebro"** detrás de las predicciones en tu aplicación Streamlit. Su influencia es directa y vital:

* **Fundamento de las Predicciones:** Proporciona los modelos matemáticos entrenados que la aplicación Streamlit usa para calcular los pesos de la carne, vísceras y caparazón a partir del peso total ingresado por el usuario.
* **Garantía de la Calidad del Modelo:** Al aplicar la transformación Box-Cox, el script asegura que los modelos de regresión lineal se ajusten mejor a los supuestos estadísticos, lo que resulta en **predicciones más precisas y fiables**. Esto se traduce en una mejor "experiencia de usuario" y mayor confianza en los resultados de la aplicación.
* **Eficiencia de la Aplicación:** Al pre-entrenar y guardar los modelos, la aplicación Streamlit no necesita ejecutar el entrenamiento cada vez que se inicia o se realiza una predicción, lo que la hace **rápida y eficiente**.
* **Transparencia y Evaluación:** La captura y guardado de métricas permite una **evaluación clara del rendimiento** del modelo, lo que es esencial para el desarrollo y mejora continua del proyecto. Se puede saber cuán bien el modelo está prediciendo los pesos de las partes del cangrejo.
* **Adaptabilidad a Rangos Específicos:** El filtrado de datos para entrenamiento permite que el modelo se especialice en el rango de peso más relevante o común de los cangrejos, lo cual es útil si el comportamiento de los cangrejos cambia significativamente fuera de ese rango.
---